In [1]:
pip install networkx shapely geopy neo4j numpy

In [2]:
pip install osmnx

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install numpy scipy

In [4]:
# Importer les bibliothèques nécessaires
import osmnx as ox  # Bibliothèque pour récupérer et visualiser les réseaux de rues à partir d'OpenStreetMap
import networkx as nx  # Bibliothèque pour créer et gérer des réseaux/graphes complexes
from shapely.geometry import Point, LineString, MultiLineString  # Pour les opérations géométriques
from scipy.spatial import distance  # Pour les calculs de distance spatiale
from geopy.distance import geodesic  # Pour les calculs de distance géodésique (à la surface de la Terre)
from neo4j import GraphDatabase  # Pilote de base de données Neo4j
import numpy as np  # Pour les opérations numériques

C:\Users\hp\anaconda3\Lib\site-packages\pyproj\network.py:59: UserWarning: pyproj unable to set PROJ database path.
  _set_context_ca_bundle_path(ca_bundle_path)


In [7]:
pip install osmnx networkx shapely scipy geopy neo4j

Note: you may need to restart the kernel to use updated packages.


In [9]:
import numpy as np
from shapely.geometry import Point, LineString
from geopy.distance import geodesic

def calculate_distance(point1, point2):
    # Euclidean distance between two points
    return np.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

def calculate_distance_in_meters(point1, point2):
    # Distance between points in meters (using a placeholder conversion)
    return calculate_distance(point1, point2) * 1000

def add_hotel_node(G, osmid, hotel_point):
    # Add a hotel node to the graph
    print('osmid : ', osmid)
    G.add_node(osmid, x=hotel_point.x, y=hotel_point.y, osmid=osmid)
    G.nodes[osmid]['hotel'] = 'yes'

def calculate_linestring_distance_in_meters(coords):
    # Total distance along a LineString in meters
    total_distance = 0.0
    for i in range(len(coords) - 1):
        distance = geodesic(coords[i], coords[i + 1]).meters
        total_distance += distance
    return total_distance

def calculate_distance(coord1, coord2):
    # Geodesic distance between two coordinates in meters
    return geodesic(coord1, coord2).meters

def linestring_to_list(geometry):
    # Convert LineString to list of coordinates
    return list(geometry.coords) if isinstance(geometry, LineString) else None

def projection_point_on_line(x1, y1, x2, y2, x3, y3):
    # Project point C onto line AB
    ABx, ABy = x2 - x1, y2 - y1
    ACx, ACy = x3 - x1, y3 - y1
    t = (ACx * ABx + ACy * ABy) / (ABx**2 + ABy**2)
    return x1 + t * ABx, y1 + t * ABy

def find_nearest_point(hotel_node, edge):
    # Find closest point on edge to hotel node
    distance, nearest_point = float('inf'), None
    for point in list(edge.coords):
        current_distance = geodesic((hotel_node.y, hotel_node.x), (point[1], point[0])).meters
        if current_distance < distance:
            distance, nearest_point = current_distance, point
    return nearest_point


In [10]:
def process_hotel(G, osmid, hotel_point):
    # Find the nearest edge to the hotel location
    nearest_edge = ox.distance.nearest_edges(G, hotel_point.x, hotel_point.y)
    u, v = nearest_edge[:2]
    edge_data = G.get_edge_data(*nearest_edge)
    
    if 'geometry' in edge_data:
        # If edge has geometry, modify it to include the hotel point
        coords_list = []
        edge_geom = LineString(list(edge_data['geometry'].coords).copy())
        nearest_point = find_nearest_point(hotel_point, edge_geom)
        edge_coords = list(edge_geom.coords)
        
        for coord in edge_coords:
            coords_list.append(coord)
            if calculate_distance(nearest_point, coord) < 0.1:
                coords_list.append((hotel_point.x, hotel_point.y))
                break
        
        edge_geom = LineString(coords_list)
        
    else:
        # If edge has no geometry, create a line between points
        edge_geom = LineString([
            (G.nodes[u]['x'], G.nodes[u]['y']),
            projection_point_on_line(
                G.nodes[u]['x'], G.nodes[u]['y'], 
                G.nodes[v]['x'], G.nodes[v]['y'], 
                hotel_point.x, hotel_point.y
            ),
            (hotel_point.x, hotel_point.y)
        ])
    
    # Define new edge data
    new_edge_data = {
        'reversed': True,
        'length': calculate_linestring_distance_in_meters(edge_geom.coords),
        'geometry': edge_geom
    }
    
    # Add the hotel node to the graph
    add_hotel_node(G, osmid, hotel_point)
    G.add_edge(u, osmid, **new_edge_data)
    
    # If the reverse edge exists and is marked as reversed, process it as well
    if v in G and u in G[v] and G[v][u] and edge_data['reversed']:
        edge_data = G[v][u][0]
        
        if 'geometry' in edge_data:
            # Adjust edge geometry if it has geometry data
            coords_list = []
            edge_geom = LineString(list(edge_data['geometry'].coords).copy())
            nearest_point = find_nearest_point(hotel_point, edge_geom)
            edge_coords = list(edge_geom.coords)
            
            for coord in edge_coords:
                coords_list.append(coord)
                if calculate_distance(nearest_point, coord) < 0.1:
                    coords_list.append((hotel_point.x, hotel_point.y))
                    break
            
            edge_geom = LineString(coords_list)
        
        else:
            # If no geometry, create a simple line with the hotel point
            edge_geom = LineString([
                (G.nodes[u]['x'], G.nodes[u]['y']),
                projection_point_on_line(
                    G.nodes[u]['x'], G.nodes[u]['y'], 
                    G.nodes[v]['x'], G.nodes[v]['y'], 
                    hotel_point.x, hotel_point.y
                ),
                (hotel_point.x, hotel_point.y)
            ])
    
    # Add the modified reverse edge to the graph
    new_edge_data = {
        'reversed': True,
        'length': calculate_linestring_distance_in_meters(edge_geom.coords),
        'geometry': edge_geom
    }
    G.add_edge(v, osmid, **new_edge_data)


In [15]:
pip install --upgrade osmnx geopandas pyproj

Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install osmnx


Note: you may need to restart the kernel to use updated packages.


In [18]:
pip install geopandas pyproj

Note: you may need to restart the kernel to use updated packages.


In [19]:
import osmnx as ox
import geopandas as gpd

# Set the default CRS
ox.settings.default_crs = 'epsg:4326'

# Try to get the graph again
G = ox.graph_from_place('Fès, Fès-Meknès, Maroc', network_type='drive')


CRSError: Invalid projection: epsg:4326: (Internal Proj Error: proj_create: no database context specified)

In [ ]:
import osmnx as ox
print(ox.__version__)

In [ ]:
import osmnx as ox
tags = {"amenity": "pharmacy"}
pharmacies = ox.geometries_from_place("Fès, Fès-Meknès, Maroc", tags=tags)
pharmacies.head()

In [ ]:
pharmacy_points = [
    ( index[1],row['geometry'])
    for index, row in pharmacies.iterrows()
]


In [ ]:
a = 1
for i,j in pharmacy_points:
    print(a,end='->')
    process_pharmacy(G,i,j)
    a+=1

In [ ]:
import numpy as np
from shapely.geometry import LineString, Point

def project_and_insert_point(path_coords, point_coords):
    """
    Project a point onto a path and insert it at the correct position in the path's coordinates.
    
    :param path_coords: List of tuples representing path coordinates [(x1, y1), (x2, y2), ...]
    :param point_coords: Tuple representing the point's coordinates (x, y)
    :return: New list of path coordinates with the projected point inserted
    """
    path = LineString(path_coords)
    point = Point(point_coords)
    
    # Find the projected position of the point on the path
    distance = path.project(point)
    projected_point = path.interpolate(distance)
    projected_coords = (projected_point.x, projected_point.y)
    
    # Insert the projected point in the correct segment of the path
    new_path_coords = []
    inserted = False
    for i in range(len(path_coords) - 1):
        segment_start = Point(path_coords[i])
        segment_end = Point(path_coords[i + 1])
        segment = LineString([segment_start, segment_end])
        
        new_path_coords.append(path_coords[i])
        
        # Check if the projected point should be inserted in this segment
        if not inserted and segment.project(projected_point) < segment.length:
            new_path_coords.append(projected_coords)
            inserted = True
    
    new_path_coords.append(path_coords[-1])  # Add the last point of the path
    
    # If the point wasn't inserted, add it to the end
    if not inserted:
        new_path_coords.append(projected_coords)

    print(projected_coords)
    return new_path_coords

# Example usage
path = [(0, 0), (1, 2), (2, 1), (3, 3)]  # Non-linear path
point = (2, 2)

new_path = project_and_insert_point(path, point)
print("The new path with the projected point is:")
for coord in new_path:
    print(coord)

In [ ]:
from neo4j import GraphDatabase

# Function to convert OSMnx graph nodes to Neo4j data
def convert_nodes_osmnx_graph_to_neo4j(G):
    # Connect to Neo4j database
    uri = "bolt://44.195.20.104:7687"
    user = "neo4j"
    password = "vendors-visibility-silicon"
    driver = GraphDatabase.driver(uri, auth=(user, password))

    # Create a Neo4j session
    # Changed database name from "pharma" to "neo4j"
    with driver.session(database="neo4j") as session:
        # Loop through each node in the OSMnx graph
        for node in G.nodes(data=True):
            # Prepare node properties
            properties = {
                'osmid': node[0],
                'y': node[1].get('y'),
                'x': node[1].get('x')
            }

            # Check if 'pharmacy' attribute exists
            if 'pharmacy' in node[1]:
                properties['pharmacy'] = node[1]['pharmacy']

            # Build Cypher query to create the node
            query = "CREATE (a:Location {osmid: $osmid, y: $y, x: $x"
            if 'pharmacy' in properties:
                query += ", pharmacy: $pharmacy"
            query += "})"

            # Filter valid properties (exclude callable and None values)
            valid_properties = {k: v for k, v in properties.items() if not callable(v) and v is not None}

            # Execute the query with node properties
            session.run(query, **valid_properties)

    # Close the connection
    driver.close()

# Call the function to convert the graph
convert_nodes_osmnx_graph_to_neo4j(G)